In [ ]:
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

In [ ]:
import datetime
import json
import re
from typing import Annotated

from IPython.display import Image
from langchain_core.messages import ToolMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict

memory = MemorySaver()


def multiply(a: int, b: int) -> int:
    """Multiply two numbers.

    Args:
        a: The first number.
        b: The second number.

    Returns:
        int: The result of the multiplication.
    """
    return a * b


class State(TypedDict):
    messages: Annotated[list, add_messages]


builder = StateGraph(State)


llm = ChatOpenAI(model="gpt-4o-mini")
llm = llm.bind_tools(tools=[multiply])


class BasicToolNode:
    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.__name__: tool for tool in tools}

    def __call__(self, inputs: dict):
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No messages found in the input.")
        outputs = []
        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"]](**tool_call["args"])
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"],
                )
            )
        return {"messages": outputs}


def route_tools(state: State):
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    return END  # can route to another node as well


tool_node = BasicToolNode(tools=[multiply])


def chatbot(state: State):
    # The return of a node updates the state. We could update another states here,
    # like state["foo"] = "bar", if we had a "foo" state on the State definition.
    return {"messages": [llm.invoke(state["messages"])]}


builder.add_node("chatbot", chatbot)
builder.add_node("tools", tool_node)
builder.add_edge(START, "chatbot")
builder.add_edge("tools", "chatbot")
builder.add_conditional_edges("chatbot", route_tools, {"tools": "tools", END: END}),
graph = builder.compile(checkpointer=memory)


timestamp = datetime.datetime.now().isoformat()
print(f"Timestamp: {timestamp}")

timestamp = re.sub(r"[-:.]", "", timestamp)
print(f"Timestamp: {timestamp}")

config = {"configurable": {"thread_id": timestamp}}
print(f"Config: {config}")

Image(graph.get_graph().draw_mermaid_png())

In [ ]:
from langchain_core.messages import HumanMessage


def stream_graph_updates(user_input: str):

    for event in graph.stream(
        {"messages": [HumanMessage(content=user_input)]},
        config=config,
        stream_mode="values",
    ):
        for value in event.values():
            print("Assistant:", value[-1].content)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["exit", "quit", "q"]:
            print("Bye!")
            break
        stream_graph_updates(user_input)
    except Exception as e:
        print(str(e))
        user_input = "What do you know about Guilherme Gaigher Netto?"
        print("User: " + user_input)
        stream_graph_updates(user_input)

In [ ]:
snapshot = graph.get_state(config)
snapshot.values